# Imports + Data Loading

In [1]:
from __future__ import annotations
import numpy as np
from dataclasses import dataclass
from typing import Dict, Any, Tuple, List

In [ ]:
# org_seed.py
from __future__ import annotations
import numpy as np
from dataclasses import dataclass

# Minimal vocab (extend as you like)
INDUSTRIES = ["tech", "finance", "healthcare", "manufacturing", "government"]
SIZES      = ["small", "mid", "large", "enterprise"]

@dataclass
class OrgSeed:
    industry_id: int          # 0..len(INDUSTRIES)-1
    size_id: int              # 0..len(SIZES)-1
    power_distance: float     # [0,1]
    sanction_salience: float  # [0,1]
    in_group_bias: float      # [0,1]

    def as_vector(self) -> np.ndarray:
        return np.array([
            float(self.industry_id),
            float(self.size_id),
            float(self.power_distance),
            float(self.sanction_salience),
            float(self.in_group_bias)
        ], dtype=float)

def sample_org_seed(seed: int | None = None) -> OrgSeed:
    """
    Small, deterministic sampler for O = (industry, size, culture_profile).
    Culture profile components are in [0,1] and independent for simplicity.
    """
    rng = np.random.default_rng(seed)

    industry_id = int(rng.integers(0, len(INDUSTRIES)))
    size_id     = int(rng.integers(0, len(SIZES)))

    # Simple uniform draws in [0,1]; swap to Beta/Normal if you want shape.
    power_distance   = float(rng.random())
    sanction_salience= float(rng.random())
    in_group_bias    = float(rng.random())

    return OrgSeed(
        industry_id=industry_id,
        size_id=size_id,
        power_distance=power_distance,
        sanction_salience=sanction_salience,
        in_group_bias=in_group_bias
    )

def decode_org_seed_to_text(org: OrgSeed) -> str:
    """
    Human-readable description matching your schema, for LLM conditioning.
    """
    return (
        "Organization O:\n"
        f"  industry={INDUSTRIES[org.industry_id]}, size={SIZES[org.size_id]}\n"
        "  culture_profile:\n"
        f"    power_distance={org.power_distance:.2f}  # acceptance of hierarchy\n"
        f"    sanction_salience={org.sanction_salience:.2f}  # fear of rule-breaking consequences\n"
        f"    in_group_bias={org.in_group_bias:.2f}  # collectivism vs individualism\n"
    )

def from_vector(vec: np.ndarray) -> OrgSeed:
    """
    Build OrgSeed back from a 5-length vector.
    """
    if len(vec) != 5:
        raise ValueError("org vector must have length 5")
    industry_id = int(np.clip(round(vec[0]), 0, len(INDUSTRIES)-1))
    size_id     = int(np.clip(round(vec[1]), 0, len(SIZES)-1))
    pd  = float(np.clip(vec[2], 0.0, 1.0))
    ss  = float(np.clip(vec[3], 0.0, 1.0))
    igb = float(np.clip(vec[4], 0.0, 1.0))
    return OrgSeed(industry_id, size_id, pd, ss, igb)

In [ ]:

# --- example ---
if __name__ == "__main__":
    O = sample_org_seed(seed=123)
    print("Vector:", O.as_vector())
    print(decode_org_seed_to_text(O))


Vector: [0.         2.         0.05382102 0.22035987 0.18437181]
Organization O:
  industry=tech, size=large
  culture_profile:
    power_distance=0.05  # acceptance of hierarchy
    sanction_salience=0.22  # fear of rule-breaking consequences
    in_group_bias=0.18  # collectivism vs individualism



In [ ]:
RECOMMENDATION_DOMAINS = [
    "product_roadmap",
    "hiring",
    "budget",
    "compliance",
    "pricing",
    "market_entry",
    "vendor_selection",
]

@dataclass
class RecSeed:
    domain_id: int       # index in RECOMMENDATION_DOMAINS
    theta_ideal: float   # [0,1] conservative (0) -> aggressive (1)
    urgency: float       # [0,1] time pressure
    resource_need: float # [0,1] implementation cost / commitment friction

    def as_vector(self) -> np.ndarray:
        return np.array(
            [
                float(self.domain_id),
                float(self.theta_ideal),
                float(self.urgency),
                float(self.resource_need),
            ],
            dtype=float,
        )

def sample_rec_seed(seed: int | None = None) -> RecSeed:
    """
    Sample R = (domain, theta_ideal, urgency, resource_need) as a simple numeric seed.
    All continuous components in [0,1].
    """
    rng = np.random.default_rng(seed)

    domain_id = int(rng.integers(0, len(RECOMMENDATION_DOMAINS)))
    theta_ideal   = float(rng.random())  # 0 = conservative, 1 = aggressive
    urgency       = float(rng.random())  # 0 = no pressure, 1 = extreme pressure
    resource_need = float(rng.random())  # 0 = cheap, 1 = very expensive

    return RecSeed(
        domain_id=domain_id,
        theta_ideal=theta_ideal,
        urgency=urgency,
        resource_need=resource_need,
    )

def _bucket(x: float) -> str:
    """
    Simple human label for [0,1] scalar: low / medium / high.
    """
    if x < 1/3:
        return "low"
    elif x < 2/3:
        return "medium"
    else:
        return "high"

def decode_rec_seed_to_text(r: RecSeed) -> str:
    """
    Human-readable description of R, for LLM conditioning.
    """
    domain = RECOMMENDATION_DOMAINS[r.domain_id]
    return (
        "Recommendation R:\n"
        f"  domain={domain}\n"
        f"  theta_ideal={r.theta_ideal:.2f}  # 0=conservative, 1=aggressive leader cue\n"
        f"  urgency={r.urgency:.2f} ({_bucket(r.urgency)} time pressure)\n"
        f"  resource_need={r.resource_need:.2f} ({_bucket(r.resource_need)} implementation cost)\n"
    )

def rec_from_vector(vec: np.ndarray) -> RecSeed:
    """
    Build RecSeed back from a 4-length vector.
    """
    if len(vec) != 4:
        raise ValueError("recommendation vector must have length 4")

    domain_id = int(np.clip(round(vec[0]), 0, len(RECOMMENDATION_DOMAINS) - 1))
    theta_ideal   = float(np.clip(vec[1], 0.0, 1.0))
    urgency       = float(np.clip(vec[2], 0.0, 1.0))
    resource_need = float(np.clip(vec[3], 0.0, 1.0))

    return RecSeed(
        domain_id=domain_id,
        theta_ideal=theta_ideal,
        urgency=urgency,
        resource_need=resource_need,
    )

In [ ]:
if __name__ == "__main__":
    R = sample_rec_seed(seed=456)
    print("Vector:", R.as_vector())
    print(decode_rec_seed_to_text(R))

Vector: [2.         0.4476553  0.12405681 0.75484998]
Recommendation R:
  domain=budget
  theta_ideal=0.45  # 0=conservative, 1=aggressive leader cue
  urgency=0.12 (low time pressure)
  resource_need=0.75 (high implementation cost)



In [ ]:
# situation_seed.py (or add to your unified hidden_state_seeds.py)
from __future__ import annotations
import numpy as np
from dataclasses import dataclass

VISIBILITY = ["private", "public"]   # 0 or 1

@dataclass
class SituationSeed:
    theta_current: float     # [0,1] — organization’s current stance (0=conservative, 1=aggressive)
    visibility_flag: int     # 0 = private, 1 = public
    sanction_strength: float # [0,1] — intensity of enforcement
    provocation_flag: int    # 0 or 1 — whether a provocative event occurred

    def as_vector(self) -> np.ndarray:
        """
        Numeric vector representation: length 4.
        """
        return np.array(
            [
                float(self.theta_current),
                float(self.visibility_flag),
                float(self.sanction_strength),
                float(self.provocation_flag),
            ],
            dtype=float,
        )

def sample_situation_seed(seed: int | None = None) -> SituationSeed:
    """
    Sample S = (theta_current, visibility, sanction_strength, provocation_flag).
    All continuous values in [0,1].
    """
    rng = np.random.default_rng(seed)

    theta_current     = float(rng.random())   # 0 conservative ←→ 1 aggressive
    visibility_flag   = int(rng.integers(0, 2))  # 0 private, 1 public
    sanction_strength = float(rng.random())   # 0 none ←→ 1 extreme consequences
    provocation_flag  = int(rng.integers(0, 2))  # 0 no provocation, 1 active provocation

    return SituationSeed(
        theta_current=theta_current,
        visibility_flag=visibility_flag,
        sanction_strength=sanction_strength,
        provocation_flag=provocation_flag,
    )

def _bucket(x: float) -> str:
    """
    Human-friendly category for continuous values.
    """
    if x < 1/3:
        return "low"
    elif x < 2/3:
        return "medium"
    else:
        return "high"

def decode_situation_seed_to_text(s: SituationSeed) -> str:
    """
    LLM-friendly textual representation of S.
    """
    visibility = VISIBILITY[s.visibility_flag]

    return (
        "Situation S:\n"
        f"  theta_current={s.theta_current:.2f}  # org's starting stance (0=conservative, 1=aggressive)\n"
        f"  visibility={visibility}  # public increases reputational sanctioning\n"
        f"  sanction_strength={s.sanction_strength:.2f} ({_bucket(s.sanction_strength)})\n"
        f"  provocation_flag={s.provocation_flag}  # 1 indicates a triggering/challenging event\n"
    )

def situation_from_vector(vec: np.ndarray) -> SituationSeed:
    """
    Build SituationSeed back from a vector of length 4.
    """
    if len(vec) != 4:
        raise ValueError("Situation vector must have length 4")

    theta_current     = float(np.clip(vec[0], 0.0, 1.0))
    visibility_flag   = int(np.clip(round(vec[1]), 0, 1))
    sanction_strength = float(np.clip(vec[2], 0.0, 1.0))
    provocation_flag  = int(np.clip(round(vec[3]), 0, 1))

    return SituationSeed(
        theta_current=theta_current,
        visibility_flag=visibility_flag,
        sanction_strength=sanction_strength,
        provocation_flag=provocation_flag,
    )

In [ ]:
# --- tiny demo ---
if __name__ == "__main__":
    S = sample_situation_seed(seed=999)
    print("Vector:", S.as_vector())
    print(decode_situation_seed_to_text(S))


Vector: [0.77882502 0.         0.71357279 0.        ]
Situation S:
  theta_current=0.78  # org's starting stance (0=conservative, 1=aggressive)
  visibility=private  # public increases reputational sanctioning
  sanction_strength=0.71 (high)
  provocation_flag=0  # 1 indicates a triggering/challenging event



In [ ]:
from __future__ import annotations
import numpy as np
from dataclasses import dataclass

EDGE_TYPES = ["reports_to", "peer", "mentorship"]

@dataclass
class GraphSeed:
    n_agents: int               # number of nodes
    levels: np.ndarray          # shape (n,)
    edge_type: np.ndarray       # shape (n,n)

    def as_vector(self) -> np.ndarray:
        n = self.n_agents
        header = np.array([float(n)], dtype=float)
        levels_vec = self.levels.astype(float)
        edge_vec = self.edge_type.astype(float).ravel()
        return np.concatenate([header, levels_vec, edge_vec], axis=0)

def sample_graph_seed(
    seed: int | None = None,
    min_agents: int = 5,
    max_agents: int = 15,
    max_levels: int = 4,
):
    rng = np.random.default_rng(seed)
    n = int(rng.integers(min_agents, max_agents + 1))

    L = int(rng.integers(2, max_levels + 1))
    level_ids = np.arange(L)
    level_probs = (level_ids[::-1] + 1).astype(float)
    level_probs /= level_probs.sum()
    levels = rng.choice(level_ids, size=n, p=level_probs).astype(int)

    if not np.any(levels == 0):
        idx = int(rng.integers(0, n))
        levels[idx] = 0

    edge_type = -np.ones((n, n), dtype=int)

    # reports_to backbone
    for i in range(n):
        li = levels[i]
        if li == 0:
            continue
        candidates = np.where(levels < li)[0]
        if candidates.size == 0:
            candidates = np.where(levels == li)[0]
            candidates = candidates[candidates != i]
        if candidates.size > 0:
            j = int(rng.choice(candidates))
            edge_type[i, j] = 0

    # peer edges
    p_peer = 0.15
    for li in range(L):
        members = np.where(levels == li)[0]
        for a in members:
            for b in members:
                if a >= b:
                    continue
                if rng.random() < p_peer:
                    edge_type[a, b] = 1
                    edge_type[b, a] = 1

    # mentorship edges
    p_mentor = 0.10
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if levels[i] >= levels[j]:
                continue
            if edge_type[i, j] != -1:
                continue
            if rng.random() < p_mentor:
                edge_type[i, j] = 2

    return GraphSeed(n_agents=n, levels=levels, edge_type=edge_type)


def decode_graph_seed_to_text(
    g: GraphSeed,
    agent_names: list[str] | None = None
) -> str:
    """
    Human-readable graph description, suitable for LLM conditioning.
    """
    n = g.n_agents
    if agent_names is None:
        agent_names = [f"agent_{i}" for i in range(n)]

    # Levels
    lines_levels = [
        f"  {agent_names[i]}: level={g.levels[i]}"
        for i in range(n)
    ]
    levels_txt = "Levels:\n" + "\n".join("    " + line for line in lines_levels)

    # Edges
    edge_lines = []
    for i in range(n):
        for j in range(n):
            t = g.edge_type[i, j]
            if t == -1:
                continue
            edge_lines.append(
                f"  {agent_names[i]} -> {agent_names[j]}: type={EDGE_TYPES[t]}"
            )
    edges_txt = "Edges:\n" + ("\n".join("    " + line for line in edge_lines) if edge_lines else "    (none)")

    return "Authority graph G:\n" + levels_txt + "\n" + edges_txt + "\n"

In [ ]:
# --- tiny demo ---
if __name__ == "__main__":
    G = sample_graph_seed(seed=123)
    print("Vector length:", len(G.as_vector()))
    print("n_agents:", G.n_agents)
    print("levels:", G.levels)
    print(decode_graph_seed_to_text(G))

Vector length: 31
n_agents: 5
levels: [0 0 0 0 2]
Authority graph G:
Levels:
      agent_0: level=0
      agent_1: level=0
      agent_2: level=0
      agent_3: level=0
      agent_4: level=2
Edges:
      agent_4 -> agent_1: type=reports_to

